## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,54.27,91,40,2.35,scattered clouds
1,1,Pogar,RU,52.5540,33.2591,55.02,83,38,8.34,scattered clouds
2,2,Kamenka,RU,51.3223,42.7678,60.94,79,100,12.39,overcast clouds
3,3,Oblesevo,MK,41.8833,22.3339,68.95,65,60,4.61,broken clouds
4,4,Kapaa,US,22.0752,-159.3190,78.78,68,40,3.00,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature desired for your trip? "))
max_temp = float(input("What is the maximum temperature desired for your trip? "))

What is the minimum temperature desired for your trip? 75
What is the maximum temperature desired for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Kapaa,US,22.0752,-159.3190,78.78,68,40,3.00,scattered clouds
9,9,Rikitea,PF,-23.1203,-134.9692,76.55,67,15,12.57,few clouds
12,12,Mandera,KE,3.9366,41.8670,83.89,54,100,14.38,overcast clouds
14,14,Atuona,PF,-9.8000,-139.0333,80.08,68,88,14.32,overcast clouds
15,15,Tiarei,PF,-17.5333,-149.3333,80.62,76,49,16.08,scattered clouds
24,24,Puerto Ayora,EC,-0.7393,-90.3518,84.16,79,85,8.99,overcast clouds
31,31,Hilo,US,19.7297,-155.0900,75.61,81,40,6.91,scattered clouds
33,33,Veraval,IN,20.9000,70.3667,83.46,80,11,11.86,few clouds
49,49,Vaini,TO,-21.2000,-175.2000,75.36,100,75,2.30,light rain
58,58,Dingle,PH,10.9995,122.6711,77.25,87,51,3.56,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                167
City                   167
Country                166
Lat                    167
Lng                    167
Max Temp               167
Humidity               167
Cloudiness             167
Wind Speed             167
Current Description    167
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                166
City                   166
Country                166
Lat                    166
Lng                    166
Max Temp               166
Humidity               166
Cloudiness             166
Wind Speed             166
Current Description    166
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,
9,Rikitea,PF,76.55,few clouds,-23.1203,-134.9692,
12,Mandera,KE,83.89,overcast clouds,3.9366,41.8670,
14,Atuona,PF,80.08,overcast clouds,-9.8000,-139.0333,
15,Tiarei,PF,80.62,scattered clouds,-17.5333,-149.3333,
24,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,
31,Hilo,US,75.61,scattered clouds,19.7297,-155.0900,
33,Veraval,IN,83.46,few clouds,20.9000,70.3667,
49,Vaini,TO,75.36,light rain,-21.2000,-175.2000,
58,Dingle,PH,77.25,broken clouds,10.9995,122.6711,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace("",np.nan)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   159
Country                159
Max Temp               159
Current Description    159
Lat                    159
Lng                    159
Hotel Name             159
dtype: int64

In [10]:
#7a. Check dataframe
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Kapaa,US,78.78,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
9,Rikitea,PF,76.55,few clouds,-23.1203,-134.9692,Pension Maro'i
12,Mandera,KE,83.89,overcast clouds,3.9366,41.8670,Kornesh Hotel and Lodging
14,Atuona,PF,80.08,overcast clouds,-9.8000,-139.0333,Villa Enata
15,Tiarei,PF,80.62,scattered clouds,-17.5333,-149.3333,Le Rocher de Tahiti
24,Puerto Ayora,EC,84.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
31,Hilo,US,75.61,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
33,Veraval,IN,83.46,few clouds,20.9000,70.3667,Lords Inn Somnath
49,Vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort
58,Dingle,PH,77.25,broken clouds,10.9995,122.6711,SEGAYA PROPERTY


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))